In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
import pandas as pd
from dataclasses import dataclass
import json
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from llamawrapper import load_unemb_only, LlamaHelper
import seaborn as sns
from scipy.stats import bootstrap
from utils import plot_ci, plot_ci_plus_heatmap
from tqdm import tqdm

# fix random seed
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

In [3]:
target_lang = 'zh'
model_size = '13b'
out_dir = './visuals'
# Need a huggingface token for some model
hf_token = ''

latent_lang_1 = 'en'
latent_lang_2 = 'fr'
model_selected = 2
layer_num = 40



In [4]:
if model_selected == 1:
    custom_model = 'llm-jp/llm-jp-13b-v2.0'
elif model_selected == 2:
    custom_model = 'meta-llama/Llama-2-%s-hf'%model_size
elif model_selected == 3:
    custom_model = 'tokyotech-llm/Swallow-%s-hf'%model_size
elif model_selected == 4:
    custom_model = 'hfl/chinese-llama-2-13b'
elif model_selected == 5:
    custom_model = 'baichuan-inc/Baichuan2-13B-Base'
elif model_selected == 6:
    custom_model = 'sambanovasystems/SambaLingo-Arabic-Base'
elif model_selected == 7:
    custom_model = 'OpenLLM-France/Claire-Mistral-7B-0.1'
elif model_selected == 8:
    custom_model = 'croissantllm/CroissantLLMBase'

In [5]:
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")

Using device: cuda


In [6]:
if custom_model is not None:
    model = LlamaHelper(dir=custom_model, load_in_8bit=True, hf_token=hf_token)

/home/zhong/miniconda3/envs/latent/lib/python3.12/site-packages/transformers/models/auto/tokenization_auto.py:769: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/zhong/miniconda3/envs/latent/lib/python3.12/site-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
tokenizer = model.tokenizer

In [8]:
file_path = 'multi5.csv'
df = pd.read_csv(file_path)

In [9]:
def capitalizations(token_str):
    variants = [token_str.lower(), token_str.capitalize()]
    return list(set(variants))

def add_spaces_(tokens):
    return ['▁' + t for t in tokens] + tokens

def add_spaces(tokens):
    return [' ' + t for t in tokens] + tokens

def add_(tokens):
    return ['_' + t for t in tokens]

def ar_(tokens):
    return ['"' + t for t in tokens]

# Obtain all possible tokenizations of a word
def process_tokens(token_str: str, tokenizer, lang):
    final_tokens = []
    # print(token_str)
    if lang in ['en', 'fr']:
        with_capitalizations = capitalizations(token_str)
        if model_selected == 9:
            with_spaces = add_spaces(with_capitalizations)
        else:
            with_spaces = add_spaces_(with_capitalizations)
        finail = with_spaces + add_(with_capitalizations)
        finail = list(set(finail))
        for tok in finail:
            default_tokens = tokenizer.tokenize(tok)
            default_token_ids = tokenizer.convert_tokens_to_ids(default_tokens)

            if tok.startswith('_') and len(default_token_ids) > 1:
                if model_selected == 1:
                    final_tokens.append(default_token_ids[2:])
                else:
                    final_tokens.append(default_token_ids[1:])
            elif model_selected == 1 and lang == 'fr':
                final_tokens.append(default_token_ids[:])
                if default_token_ids[0] == 31:
                    final_tokens.append(default_token_ids[1:])
            else:
                final_tokens.append(default_token_ids)
    elif lang in ['ja', 'zh']:
        finail = ['▁' + token_str] + [token_str]
        for tok in finail:
            default_tokens = tokenizer.tokenize(tok)
            default_token_ids = tokenizer.convert_tokens_to_ids(default_tokens)
            final_tokens.append(default_token_ids)
            if len(default_token_ids) > 1 and model_selected!=5:
                final_tokens.append(default_token_ids[1:])
    elif lang in ['ar']:
        finail = ['▁' + token_str] + [token_str] + ['"' + token_str]
        for tok in finail:
            default_tokens = tokenizer.tokenize(tok)
            default_token_ids = tokenizer.convert_tokens_to_ids(default_tokens)
            if tok.startswith('"'):
                final_tokens.append(default_token_ids[1:])
            else:
                final_tokens.append(default_token_ids)
                if len(default_token_ids) > 1:
                    final_tokens.append(default_token_ids[1:])
    return final_tokens

In [10]:
def remove_duplicates(nested_list):
    if len(nested_list) <= 1:
        return nested_list
    return [list(x) for x in set(tuple(x) for x in nested_list)]

In [11]:
key = f'{target_lang}_prompt_masked'
# blank_prompt_translation_masked
dataset_gap = []
n_skip = 2
for idx, (idx_df, row) in tqdm(enumerate(df.iterrows())):
    prompt_template = f''
    indices = set(list(range(len(df[key])))) - set([idx])
    idx_examples = np.random.choice(list(indices), n_skip, replace=False)
    prompt_template += f'{df[key][idx_examples[0]]}\n'
    prompt_template += f'{df[key][idx_examples[1]]}\n' 

    out_token_str = row[target_lang]
    out_token_id = process_tokens(out_token_str, tokenizer, target_lang)
    out_token_str2 = row[target_lang + '1']
    out_token_id2 = process_tokens(out_token_str2, tokenizer, target_lang)
    out_token_id = out_token_id + out_token_id2
    out_token_id = remove_duplicates(out_token_id)

    latent_token_str_1 = row[latent_lang_1]
    latent_token_id_1 = process_tokens(latent_token_str_1, tokenizer, latent_lang_1)
    latent_token_str_2 = row[latent_lang_2]
    latent_token_id_2 = process_tokens(latent_token_str_2, tokenizer, latent_lang_2)

    latent_token_str_11 = row[latent_lang_1 + '1']
    latent_token_id_11 = process_tokens(latent_token_str_11, tokenizer, latent_lang_1)
    latent_token_str_22 = row[latent_lang_2 + '1']
    latent_token_id_22 = process_tokens(latent_token_str_22, tokenizer, latent_lang_2)

    latent_token_id_1 = latent_token_id_1 + latent_token_id_11
    latent_token_id_2 = latent_token_id_2 + latent_token_id_22
    latent_token_id_1 = remove_duplicates(latent_token_id_1)
    latent_token_id_2 = remove_duplicates(latent_token_id_2)

    prompt = row[key].split(":")[0]+": \""
    dataset_gap.append({
        'prompt': prompt_template + prompt,
        'out_token_id': out_token_id,
        'out_token_str': out_token_str,
        'latent_token_id_1': latent_token_id_1,
        'latent_token_str_1': latent_token_str_1,
        'latent_token_id_2': latent_token_id_2,
        'latent_token_str_2': latent_token_str_2,

    })

0it [00:00, ?it/s]

166it [00:00, 776.78it/s]


In [12]:
len(dataset_gap)

166

In [13]:
df_gap = pd.DataFrame(dataset_gap)

In [14]:
os.makedirs(f'{os.path.join(out_dir, custom_model)}/cloze', exist_ok=True)
df_gap.to_csv(f'{os.path.join(out_dir, custom_model)}/cloze/{target_lang}_dataset.csv', index=False)

In [15]:
def compute_sequence_probabilities(prompt, tokenizer, token_sequences, model, where, model_selected, layer_num):

    prompt_ids = tokenizer.encode(prompt, return_tensors='pt')
    if model_selected == 1:
        prompt_ids = prompt_ids[:, :-1]
    total_probabilities = torch.zeros(layer_num)
    for token_sequence in token_sequences:
        sequence_probabilities = torch.ones(layer_num)
        current_input_ids = prompt_ids

        with torch.no_grad():
            for token_id in token_sequence:
                outputs = model(current_input_ids, output_hidden_states=True)
                layer_probabilities = torch.ones(layer_num)

                for layer_id in range(layer_num):
                    layer_output = model.model.layers[layer_id].output

                    if where == 1:
                        normed = model.model.norm(layer_output)
                    elif where == 2:
                        normed = model.model.layers[layer_id].post_attention_layernorm(layer_output)

                    logits = model.lm_head(normed)
                    token_probabilities = logits[:, -1, :].softmax(dim=-1).detach().cpu()[:, token_id]
                    layer_probabilities[layer_id] = token_probabilities

                sequence_probabilities *= layer_probabilities
                new_input_ids = torch.tensor([[token_id]], device=current_input_ids.device)
                current_input_ids = torch.cat([current_input_ids, new_input_ids], dim=1)
        
        total_probabilities += sequence_probabilities

    return total_probabilities

In [16]:
os.makedirs(f'{os.path.join(out_dir, custom_model)}/cloze', exist_ok=True)

In [17]:
dataset_gap_sub=dataset_gap[0:20]

In [ ]:
latent_token_probs_1 = []
latent_token_probs_2 = []
out_token_probs = []
equal = 0
# in_token_probs = []

end = -2 if model_selected == 1 else -1
for idx, d in tqdm(enumerate(dataset_gap)):
    out_probs = compute_sequence_probabilities(prompt = d['prompt'], tokenizer = tokenizer, token_sequences = d['out_token_id'], model = model.model, where = 1, model_selected = model_selected, layer_num = layer_num)
    if target_lang == latent_lang_1:
        equal = 1
        latent_probs_1 = out_probs
    else:
        latent_probs_1 = compute_sequence_probabilities(prompt = d['prompt'], tokenizer = tokenizer, token_sequences = d['latent_token_id_1'], model = model.model, where = 1, model_selected = model_selected, layer_num = layer_num)
    if target_lang == latent_lang_2:
        equal = 2
        latent_probs_2 = out_probs
    elif latent_lang_1 == latent_lang_2:
        latent_probs_2 = latent_probs_1
    else:
        latent_probs_2 = compute_sequence_probabilities(prompt = d['prompt'], tokenizer = tokenizer, token_sequences = d['latent_token_id_2'], model = model.model, where = 1, model_selected = model_selected, layer_num = layer_num)
    # in_probs = compute_sequence_probabilities(prompt = d['prompt'], tokenizer = tokenizer, token_sequences = d['in_token_id'], llama = model, unemb = unemb, end = end)

    latent_token_probs_1.append(latent_probs_1)
    latent_token_probs_2.append(latent_probs_2)
    out_token_probs.append(out_probs)
    # in_token_probs.append(in_probs)

latent_token_probs_1 = torch.stack(latent_token_probs_1)
latent_token_probs_2 = torch.stack(latent_token_probs_2)
out_token_probs = torch.stack(out_token_probs)
# in_token_probs_11 = torch.stack(in_token_probs)
np.savez(f'{os.path.join(out_dir, custom_model)}/cloze/{model_size}_{target_lang}_probas.npz',
         latent_token_probs_1=latent_token_probs_1,
         latent_token_probs_2=latent_token_probs_2,
         out_token_probs=out_token_probs)

In [ ]:
def plot_ci_mk2(ax, data, label, color='blue', linestyle='-', tik_step=4, layer_num = 40):

    data = np.array(data)
    for i in range(0, layer_num+1, tik_step):
        ax.axvline(i, color='black', linestyle='--', alpha=0.2, linewidth=1)
    mean = data.mean(axis=0)
    std = data.std(axis=0)
    std_err = std / np.sqrt(data.shape[0])

    data_ci = {
        'x': np.arange(data.shape[1]) + 1,
        'y': mean,
        'y_upper': mean + 1.96 * std_err,
        'y_lower': mean - 1.96 * std_err,
    }

    df = pd.DataFrame(data_ci)
    ax.plot(df['x'], df['y'], label=label, color=color, linestyle=linestyle)
    ax.fill_between(df['x'], df['y_lower'], df['y_upper'], color=color, alpha=0.3)

def plot_ci_plus_heatmap_mk2(latent_token_probs_1, latent_token_probs_2, out_token_probs, latent_lang_1, latent_lang_2, target_lang, color='blue', tik_step=4, equal = 0, layer_num = 40):

    fig, ax = plt.subplots(figsize=(6, 4))
    
    if equal == 1:
        plot_ci_mk2(ax, latent_token_probs_1, f'{latent_lang_1}_target', color='tab:orange', tik_step=tik_step, layer_num=layer_num)
    else: 
        plot_ci_mk2(ax, latent_token_probs_1, f'{latent_lang_1}_latent', color='tab:orange', tik_step=tik_step, layer_num=layer_num)
    if equal == 2:
        plot_ci_mk2(ax, latent_token_probs_2, f'{latent_lang_2}_target', color='tab:red', tik_step=tik_step, layer_num=layer_num)
    else:
        plot_ci_mk2(ax, latent_token_probs_2, f'{latent_lang_2}_latent', color='tab:red', tik_step=tik_step, layer_num=layer_num)
    
    if equal == 0:
        plot_ci_mk2(ax, out_token_probs, f'{target_lang}_target', color='tab:blue', tik_step=tik_step)
    
    ax.set_xlabel('layer')
    ax.set_ylabel('probability')
    
    ax.set_xlim(0, layer_num)
    ax.set_ylim(0, 1)
    
    ax.set_xticks(np.arange(0, layer_num+1, step=tik_step))
    ax.set_yticks([0, 0.5, 1.0])
    ax.legend(loc='upper left')

    plt.tight_layout()
    return fig, ax


fig, ax = plot_ci_plus_heatmap_mk2(latent_token_probs_1, latent_token_probs_2, out_token_probs, latent_lang_1, latent_lang_2, target_lang, color='tab:orange', tik_step=5, equal=equal, layer_num=layer_num)

plt.savefig(f'{os.path.join(out_dir, custom_model)}/cloze/{model_size}_{target_lang}_probas_ent.pdf', dpi=300, bbox_inches='tight')
plt.show()


In [18]:
import torch
import torch.nn.functional as F

def custom_beam_search(model, tokenizer, device, prompt, num_beams=5, max_length=50, layer_id=39):
    prompt_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)

    all_candidates = []
    all_probabilities = []

    for step in range(max_length):
        if not all_candidates:
            with torch.no_grad():
                outputs = model(prompt_ids)
                layer_output = model.model.layers[layer_id].output
                normed = model.model.norm(layer_output)
                logits = model.lm_head(normed)
                probs = F.softmax(logits[:, -1, :], dim=-1)
                top_probs, top_indices = torch.topk(probs, num_beams, dim=1)
            for i in range(num_beams):
                all_candidates.append([top_indices[0, i].item()])
                all_probabilities.append(top_probs[0, i].item())
        else:
            new_candidates = []
            new_probabilities = []

            for i in range(len(all_candidates)):
                candidate_ids = prompt_ids.clone()
                candidate_ids = torch.cat([candidate_ids.squeeze(0), torch.tensor(all_candidates[i], device=device)], dim=0).unsqueeze(0)
                with torch.no_grad():
                    outputs = model(candidate_ids)
                    layer_output = model.model.layers[layer_id].output
                    normed = model.model.norm(layer_output)
                    logits = model.lm_head(normed)
                    probs = F.softmax(logits[:, -1, :], dim=-1)
                    top_probs, top_indices = torch.topk(probs, num_beams, dim=1)

                for j in range(num_beams):
                    new_candidates.append(all_candidates[i] + [top_indices[0, j].item()])
                    new_probabilities.append(all_probabilities[i] * top_probs[0, j].item())

            all_candidates = new_candidates
            all_probabilities = new_probabilities


            top_indices = torch.topk(torch.tensor(all_probabilities), num_beams, largest=True).indices
            all_candidates = [all_candidates[i] for i in top_indices]
            all_probabilities = [all_probabilities[i] for i in top_indices]

    final_results = [tokenizer.decode(candidate) for candidate in all_candidates]
    return list(zip(final_results, all_candidates, all_probabilities))


In [38]:
# layer_idx = 32
# num_beams = 8
# n = 5
# max_length = 1
# prompt = dataset_gap[n]['prompt']
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# # prompt = dataset[4].enTja
# results = custom_beam_search(model=model.model, tokenizer=tokenizer, device=device, prompt=prompt, num_beams=num_beams, max_length=max_length, layer_id=layer_idx)
# # results = generate_intermediate_text_with_beam_search_hidden(model=model.model, tokenizer=tokenizer, device=device, layer_idx=39, prompt=prompt, num_beams=num_beams, max_length=max_length)
# for result, id, probability in results:
#     print(f"Sequence: {result}, Ids: {id}, Probability: {probability:.4f}")
# dataset_gap[n]

Sequence: 银行, Ids: [3001], Probability: 0.1475
Sequence: 金融机构, Ids: [19177], Probability: 0.0479
Sequence: 银行的, Ids: [35347], Probability: 0.0367
Sequence: 存款, Ids: [11243], Probability: 0.0314
Sequence: 商业银行, Ids: [30453], Probability: 0.0273
Sequence: 银行存款, Ids: [59412], Probability: 0.0223
Sequence: 在银行, Ids: [82798], Probability: 0.0190
Sequence: ATM, Ids: [71456], Probability: 0.0168


{'prompt': '"__"は、何かにかかるお金です。答え: "費用"\n"__"は、何かをするためのチャンスです。答え: "機会"\n"__"は、お金を預けたり借りたりする場所です。答え: "',
 'out_token_id': [[92311, 101578, 92406],
  [3729, 100486, 98854],
  [92311, 3729, 100486, 98854],
  [101578, 92406],
  [92406],
  [100486, 98854]],
 'out_token_str': '銀行',
 'latent_token_id_1': [[68498, 13968],
  [6592, 13968],
  [6565],
  [5893],
  [14503, 13968],
  [28295, 13968],
  [30321],
  [24563]],
 'latent_token_str_1': 'bank',
 'latent_token_id_2': [[19177], [3001], [92311, 19177], [92311, 3001]],
 'latent_token_str_2': '银行'}

In [36]:
# for i in dataset_gap[0]['latent_token_id_2']:
#     alternative_decoded_strings=tokenizer.decode(i)
#     print(alternative_decoded_strings)

 电话
手机
 手机
电话
